In [42]:
import cdms2 as cdms
import MV2 as MV
import genutil,cdutil,cdtime
from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
import glob
import string

import numpy as np
import matplotlib.pyplot as plt

import sys,os
sys.path.append("/Users/kmarvel/Google Drive/python-utils")

import CMIP5_tools as cmip5
import DA_tools
import Plotting

cmip_directory = "/Users/kmarvel/Google Drive/Sahel/DATA/FIX/"
daily_directory = "/Volumes/SahelData/Sahel_daily/"

In [43]:
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

In [87]:

def get_rcp(X):
    X.getAxis(0).designateTime()
    if X.getTime().units.find("days")<=0:
        newax = cdms.createAxis(X.getTime()[:]*30)
        newax.units = 'days since 1960-1-1'
        newax.designateTime()
        newax.setCalendar(cdtime.Calendar360)
        newax.id='time'
        X.setAxis(0,newax)
    cdutil.setTimeBoundsMonthly(X)
    
    return X(time=('2006-1-1','2099-12-31'))

def get_historical(X):

    X.getAxis(0).designateTime()
    if X.getTime().units.find("days")<=0:
        newax = cdms.createAxis(X.getTime()[:]*30)
        newax.units = 'days since 1960-1-1'
        newax.designateTime()
        newax.setCalendar(cdtime.Calendar360)
        newax.id='time'
    X.setAxis(0,newax)
    
    cdutil.setTimeBoundsMonthly(X)
    
    return X(time=('1900-1-1','2005-12-31'))

In [155]:
fmonthly=cdms.open(cmip_directory+"cmip5.sahel_precip.historical-rcp85.nc")
east=fmonthly("pr_CE")
fmonthly.close()
splicedmodels=cmip5.models(east)
okmodels=[]
okrips=[]
rcpok=[]
histok=[]
for splicedmodel in splicedmodels:
    hmod,rmod=splicedmodel.split(" SPLICED WITH ")
    modelid=hmod.split(".")[1]
    rip=hmod.split(".")[3]
    #find the corresponding files in both historical and rcp
    histfileeast=insensitive_glob(daily_directory+"historical/"+modelid+"_*"+rip+"*East*")
    if histfileeast!=[]:
        histok+=[histfileeast[0]]
        
    #westhistok=len(insensitive_glob(daily_directory+"historical/"+modelid+"*"+rip+"*West*"))==1
    rcpfileeast=insensitive_glob(daily_directory+"rcp85/"+modelid+"*"+rip+"*East*")
    if rcpfileeast != []:
        rcpok+=[rcpfileeast[0]]
    if ((histfileeast != []) and  (rcpfileeast != [])):   
        okmodels+=[modelid+"*"+rip]

reallyok=[]
for x in okmodels:


    histfileeast=insensitive_glob(daily_directory+"historical/"+x+"*East*")[0]
    rcpfileeast=insensitive_glob(daily_directory+"rcp85/"+x+"*East*")[0]
    if ((histfileeast!=[]) and  (rcpfileeast != [])):
        fh=cdms.open(histfileeast)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        fr=cdms.open(rcpfileeast)
        rcpdata=get_rcp(fr("flag"))
        splicetest=MV.concatenate((historicaldata,rcpdata),axis=0)
        fr.close()
        if len(splicetest)== 2400:
            reallyok+=[x]
EAST=MV.zeros((len(reallyok),200*12)) 
WEST=MV.zeros((len(reallyok),200*12)) 
i=0
for x in reallyok:
    histfileeast=insensitive_glob(daily_directory+"historical/"+x+"*East*")[0]
    rcpfileeast=insensitive_glob(daily_directory+"rcp85/"+x+"*East*")[0]
    if ((histfileeast!=[]) and  (rcpfileeast != [])):
        fh=cdms.open(histfileeast)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        fr=cdms.open(rcpfileeast)
        rcpdata=get_rcp(fr("flag"))
        splicetest=MV.concatenate((historicaldata,rcpdata),axis=0)
        fr.close()
        EAST[i]=splicetest
        i+=1
i=0
for x in reallyok:
    histfilewest=insensitive_glob(daily_directory+"historical/"+x+"*West*")[0]
    rcpfilewest=insensitive_glob(daily_directory+"rcp85/"+x+"*West*")[0]
    if ((histfilewest!=[]) and  (rcpfilewest != [])):
        fh=cdms.open(histfilewest)
        historicaldata=get_historical(fh("flag"))
        fh.close()
        fr=cdms.open(rcpfilewest)
        rcpdata=get_rcp(fr("flag"))
        splicetest=MV.concatenate((historicaldata,rcpdata),axis=0)
        fr.close()
        WEST[i]=splicetest
        i+=1
        
        
   

(31, 2400)

In [150]:
reallyok

['CCSM4*r1i1p1',
 'CCSM4*r2i1p1',
 'CESM1-CAM5*r1i1p1',
 'CMCC-CM*r1i1p1',
 'CMCC-CMS*r1i1p1',
 'CNRM-CM5*r1i1p1',
 'CSIRO-Mk3-6-0*r1i1p1',
 'CanESM2*r1i1p1',
 'CanESM2*r2i1p1',
 'CanESM2*r3i1p1',
 'CanESM2*r4i1p1',
 'CanESM2*r5i1p1',
 'GFDL-CM3*r1i1p1',
 'HadGEM2-AO*r1i1p1',
 'IPSL-CM5A-LR*r1i1p1',
 'IPSL-CM5A-LR*r2i1p1',
 'IPSL-CM5A-LR*r3i1p1',
 'IPSL-CM5A-LR*r4i1p1',
 'IPSL-CM5A-MR*r1i1p1',
 'IPSL-CM5B-LR*r1i1p1',
 'MIROC-ESM-CHEM*r1i1p1',
 'MIROC-ESM*r1i1p1',
 'MIROC5*r1i1p1',
 'MIROC5*r2i1p1',
 'MIROC5*r3i1p1',
 'MPI-ESM-LR*r1i1p1',
 'MPI-ESM-LR*r2i1p1',
 'MPI-ESM-LR*r3i1p1',
 'MPI-ESM-MR*r1i1p1',
 'NorESM1-M*r1i1p1',
 'inmcm4*r1i1p1']